In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import classification_report
import re
from collections import Counter

In [3]:
data_df = pd.read_csv('dataset\prachathai-67k.csv' )
data_df.head()

,url,date,title,body_text,labels
0,https://prachatai.com/print/42,2004-08-24 14:31,"นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ""",ประชาไท --- 23 ส.ค.2547 นักวิจัยฯ ชี้นโยบายจี...,"['ข่าว', 'สิ่งแวดล้อม']"
1,https://prachatai.com/print/41,2004-08-24 14:22,ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ,ประชาไท- 23 ส.ค.2547 นักวิชาการ ภาคประชาชน จ...,"['ข่าว', 'สิ่งแวดล้อม']"
2,https://prachatai.com/print/43,2004-08-24 15:17,จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย,นโยบายที่อนุญาตให้ปลูกร่วมกับพืชอื่นได้นั้นถื...,"['ข่าว', 'สิ่งแวดล้อม']"
3,https://prachatai.com/print/45,2004-08-24 15:58,ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...,ประชาไท -- 23 ส.ค. 47 ขาประจำทักษิณ ฟันธง ฟอง...,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล..."
4,https://prachatai.com/print/47,2004-08-24 16:10,กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง,ประชาไท-23 ส.ค.47 คณะกรรมการอนุรักษ์ ผืนป่าตะ...,"['ข่าว', 'สิ่งแวดล้อม']"


In [4]:
data_df['combined_text'] = data_df['title'] + ' ' + data_df['body_text']
data_df = data_df.drop(columns=['url', 'date', 'title', 'body_text'])
data_df.head()

,labels,combined_text
0,"['ข่าว', 'สิ่งแวดล้อม']","นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ"" ประชาไท..."
1,"['ข่าว', 'สิ่งแวดล้อม']",ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ ประชาไท- 23 ส.ค...
2,"['ข่าว', 'สิ่งแวดล้อม']",จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย นโยบายที่อนุญาตใ...
3,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล...",ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...
4,"['ข่าว', 'สิ่งแวดล้อม']",กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง ประชาไท-23 ส.ค....


In [5]:
data_df.shape

(67889, 2)

In [6]:
min_label_count = 10
label_counts = data_df['labels'].value_counts()
labels_to_keep = label_counts[label_counts >= min_label_count].index.tolist()
filtered_data = data_df[data_df['labels'].isin(labels_to_keep)]

In [7]:
filtered_data.shape

(22381, 2)

Tokenization by Pythai

In [8]:
filtered_data['tokenized_text'] = filtered_data['combined_text'].apply(lambda x: ' '.join(word_tokenize(x, engine='newmm')))

C:\Users\Acer\AppData\Local\Temp\ipykernel_18176\3974605809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['tokenized_text'] = filtered_data['combined_text'].apply(lambda x: ' '.join(word_tokenize(x, engine='newmm')))


Stop word จ้า

In [9]:
import re
from pythainlp.corpus import thai_stopwords

# กำหนด stop words ภาษาไทย
stopwords = list(thai_stopwords())

# ฟังก์ชันลบสัญลักษณ์พิเศษ, ตัวเลข และ stop words
def clean_text(text):
    # ลบสัญลักษณ์พิเศษและตัวเลขทั้งหมด
    text = re.sub(r'[\"\'\(\)\-\.!/,0-9]', '', text)
    
    # ลบ stop words
    return ' '.join([word for word in text.split() if word not in stopwords])

# ตัวอย่างการใช้งาน
filtered_data['cleaned_text'] = filtered_data['tokenized_text'].apply(clean_text)

C:\Users\Acer\AppData\Local\Temp\ipykernel_18176\300057925.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['cleaned_text'] = filtered_data['tokenized_text'].apply(clean_text)


In [10]:
filtered_data.head()

,labels,combined_text,tokenized_text,cleaned_text
0,"['ข่าว', 'สิ่งแวดล้อม']","นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ"" ประชาไท...","นักวิจัย หนุน "" แม้ว "" เปิด "" จีเอ็มโ...",นักวิจัย หนุน แม้ว จีเอ็มโอ ประชา ไท สค นักวิจ...
1,"['ข่าว', 'สิ่งแวดล้อม']",ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ ประชาไท- 23 ส.ค...,ภาค ประชาชน ต้าน เปิด เสรี จีเอ็มโอ ประชา ไท...,ประชาชน ต้าน เสรี จีเอ็มโอ ประชา ไท สค นักวิชา...
2,"['ข่าว', 'สิ่งแวดล้อม']",จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย นโยบายที่อนุญาตใ...,จุฬาฯ ห่วง จีเอ็มโอ ลาม ข้าว ไทย นโยบาย ท...,จุฬาฯ ห่วง จีเอ็มโอ ลาม ข้าว ไทย นโยบาย อนุญาต...
4,"['ข่าว', 'สิ่งแวดล้อม']",กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง ประชาไท-23 ส.ค....,กอ ต. เสนอ เลิก ถนน คลอง ลาน - อุ้มผาง ประชา...,กอ ต เสนอ เลิก ถนน คลอง ลาน อุ้มผาง ประชา ไท ส...
5,"['ข่าว', 'สิ่งแวดล้อม']",สำรวจเส้นทางอดีตถนนสายความมั่งคง คลองลาน - อุ้...,สำรวจ เส้นทาง อดีต ถนน สาย ความ มั่ง คง คลอง...,สำรวจ เส้นทาง ถนน สาย มั่ง คลอง ลาน อุ้มผาง ภา...


จะเห็น step by step ก่อนจะ train

In [35]:
from collections import Counter

# นำข้อความที่ผ่านการลบ stopwords มารวมกันทั้งหมดแล้วตัดคำแยกออก
all_words = ' '.join(filtered_data['cleaned_text']).split()

# นับจำนวนคำที่พบ
word_counts = Counter(all_words)

# จัดเรียงคำตามจำนวนการปรากฏมากไปน้อย
sorted_word_counts = word_counts.most_common(20)

# แสดง 20 คำที่ปรากฏบ่อยที่สุด
for word, count in sorted_word_counts:
    print(f"'{word}': {count} ครั้ง")

'คน': 129463 ครั้ง
'ไทย': 110235 ครั้ง
'ประชาชน': 79855 ครั้ง
'เรื่อง': 76078 ครั้ง
'ปี': 73521 ครั้ง
'ประเทศ': 69548 ครั้ง
'รัฐบาล': 67816 ครั้ง
'ทำ': 56276 ครั้ง
'ที่จะ': 51267 ครั้ง
'พื้นที่': 51217 ครั้ง
'แรงงาน': 48839 ครั้ง
'วันที่': 46071 ครั้ง
'รัฐ': 42883 ครั้ง
'จังหวัด': 42485 ครั้ง
'สังคม': 39857 ครั้ง
'ผม': 38711 ครั้ง
'สร้าง': 37913 ครั้ง
'สิทธิ': 34873 ครั้ง
'ปัญหา': 34033 ครั้ง
'บาท': 33689 ครั้ง


ขั้นตอนการเตรียม train model

In [90]:
max_words = 10000  # จำนวนคำสูงสุดที่จะแปลง
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(filtered_data['cleaned_text'])
sequences = tokenizer.texts_to_sequences(filtered_data['cleaned_text'])


In [103]:
max_len = 500  # ความยาวสูงสุดของข้อความ
data = pad_sequences(sequences, maxlen=max_len)

In [104]:
# label_encoder = LabelEncoder()
# encoded_labels = label_encoder.fit_transform(filtered_data['labels'])

from sklearn.preprocessing import MultiLabelBinarizer

# แปลงป้ายกำกับจากลิสต์ให้เป็นฟอร์แมตหลายร้อน (multi-hot encoded)
mlb = MultiLabelBinarizer()
encoded_labels = mlb.fit_transform(filtered_data['labels'])

print(mlb.classes_)


['กวีประชาไท' 'การชุมนุมเดือนมีนาคม - พฤษภาคม 2553' 'การเมือง'
 'ขีดเส้นใต้' 'ข่าว' 'ข้อเรียกร้องเปลี่ยนแปลงสภาพการจ้าง' 'คนเสื้อแดง'
 'คุณภาพชีวิต' 'ตลาดแรงงาน' 'ต่างประเทศ' 'ธนาคารกรุงเทพ' 'ธุลีดาวหาง'
 'นักปรัชญาชายขอบ' 'บทความ' 'บริษัท จอร์จี้ แอนด์ ลู จำกัด' 'ประกันสังคม'
 'ประยุทธ์ จันทร์โอชา' 'พนักงานธนาคาร' 'รอบโลกแรงงาน'
 'รายการคืนความสุขให้คนในชาติ' 'รายงานพิเศษ' 'วัฒนธรรม' 'ว่างงาน'
 'สถานการณ์แรงงานประจำสัปดาห์' 'สปสช.' 'สหภาพแรงงานธนาคารกรุงเทพ'
 'สหภาพแรงงานอุตสาหกรรมสิ่งทอและตัดเย็บเสื้อผ้าสัมพันธ์' 'สังคม'
 'สันกำแพง' 'สิทธิมนุษยชน' 'สิ่งแวดล้อม' 'สุขภาพ' 'สุขุมพจน์ คำสุขุม'
 'เชียงใหม่' 'เลิกจ้าง' 'เวฬุ เวสารัช' 'เศรษฐกิจ' 'แรงงาน']


In [106]:
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)


เริ่ทขั้นตอนทำ LSTM

In [107]:
# model = Sequential()
# model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
# model.add(LSTM(128, return_sequences=False))  # LSTM layer แทน CNN

# model.add(Dense(128, activation='relu'))
# model.add(Dense(len(label_encoder.classes_), activation='softmax'))


# model = Sequential()
# model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
# model.add(LSTM(128, return_sequences=False))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(len(mlb.classes_), activation='sigmoid'))  # ใช้ sigmoid สำหรับหลายป้ายกำกับ


model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(128, activation='relu'))
model.add(Dense(len(mlb.classes_), activation='sigmoid'))  # ใช้ sigmoid สำหรับหลายป้ายกำกับ



In [108]:
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[BinaryAccuracy(), Precision(), Recall()])


In [109]:
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)


In [114]:
# history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
560/560 [==============================] - 29s 52ms/step - loss: 0.0098 - binary_accuracy: 0.9965 - precision_10: 0.9691 - recall_10: 0.9633 - val_loss: 0.0801 - val_binary_accuracy: 0.9818 - val_precision_10: 0.8399 - val_recall_10: 0.8101
Epoch 2/10
560/560 [==============================] - 32s 57ms/step - loss: 0.0084 - binary_accuracy: 0.9970 - precision_10: 0.9735 - recall_10: 0.9691 - val_loss: 0.0901 - val_binary_accuracy: 0.9816 - val_precision_10: 0.8364 - val_recall_10: 0.8094
Epoch 3/10
560/560 [==============================] - 31s 56ms/step - loss: 0.0069 - binary_accuracy: 0.9975 - precision_10: 0.9769 - recall_10: 0.9744 - val_loss: 0.0974 - val_binary_accuracy: 0.9821 - val_precision_10: 0.8505 - val_recall_10: 0.8033
Epoch 4/10
560/560 [==============================] - 33s 59ms/step - loss: 0.0061 - binary_accuracy: 0.9977 - precision_10: 0.9790 - recall_10: 0.9774 - val_loss: 0.0949 - val_binary_accuracy: 0.9817 - val_precision_10: 0.8385 - val_recall_10:

In [115]:
# y_pred_probs = model.predict(X_test)
# y_pred = y_pred_probs.argmax(axis=-1)

# # ทำนายความน่าจะเป็นและทำการแปลงเป็นค่าไบนารี
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

140/140 [==============================] - 3s 19ms/step


In [116]:


# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# # สร้างรายงานการจำแนก
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

                                                       precision    recall  f1-score   support

                                           กวีประชาไท       0.67      0.20      0.31        10
                  การชุมนุมเดือนมีนาคม - พฤษภาคม 2553       0.00      0.00      0.00         1
                                             การเมือง       0.80      0.85      0.82      2859
                                           ขีดเส้นใต้       0.19      0.09      0.12       140
                                                 ข่าว       0.92      0.96      0.94      4041
                   ข้อเรียกร้องเปลี่ยนแปลงสภาพการจ้าง       1.00      1.00      1.00         3
                                           คนเสื้อแดง       0.00      0.00      0.00         4
                                          คุณภาพชีวิต       0.91      0.61      0.73       114
                                           ตลาดแรงงาน       1.00      0.33      0.50         3
                                           ต่างปร

c:\Users\Acer\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Acer\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Acer\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Acer\anaconda3\env